In [5]:
from exp_loader import ExperimentLoader
from IPython.display import display
import pandas as pd

exp = ExperimentLoader("result")
exp.clean_saved_models(1, dry_run=False)
result = exp.results

In [ ]:
GROUPBY_KEY = ["embeddings_name", "data_name"]
METRICS_NAME = [
    "train_loss", "val_loss", "train_precision", "train_recall", 
    "train_f1", "val_precision", "val_recall", "val_f1"
]
CATEGORICAL_NAMES = ["chunk_size", "n_epoch"]
TO_REMOVE = ["n_train", "n_val", "ns_start", "ns_end", "repeat", "epoch"]
def best_model_pre_repeat(result: pd.DataFrame):
    tmp = []
    for k, df in result.groupby(["experiment_id"] + GROUPBY_KEY + ["repeat"]):
        df: pd.DataFrame
        tmp.append(
            df.sort_values(by="val_loss")[:1]
        )
    return pd.concat(tmp)


def get_metrics(result: pd.DataFrame) -> pd.DataFrame:
    df = best_model_pre_repeat(result)
    df.drop(columns=["experiment_id"])
    return df[GROUPBY_KEY + METRICS_NAME]

metrics = get_metrics(result)
print(metrics.groupby(["data_name", "embeddings_name"])[['val_loss']].count())

metrics.groupby(["data_name", "embeddings_name"]).mean()[['val_loss', 'val_recall']].sort_values(by=["val_loss", "val_recall"])
metrics.boxplot(column=["val_loss"], by=["data_name", "embeddings_name"], rot=90)
metrics.boxplot(column=["val_recall"], by=["data_name", "embeddings_name"], rot=90)
